In [1]:
from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .appName('ddam_project')
         .config('spark.some.config.option','some-value')
         .getOrCreate()
         )

In [2]:
# read file from hdfs and infer schema
df_cleaned = spark.read.csv("hdfs://masterbig-1.itc.unipi.it:54310/user/student18/df_cleaned.csv", header = True, inferSchema = True)
df_cleaned.printSchema()

root
 |-- id: long (nullable = true)
 |-- Hotel_Address: string (nullable = true)
 |-- Additional_Number_of_Scoring: integer (nullable = true)
 |-- Review_Date: string (nullable = true)
 |-- Average_Score: double (nullable = true)
 |-- Hotel_Name: string (nullable = true)
 |-- Reviewer_Nationality: string (nullable = true)
 |-- Review_Total_Negative_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews: integer (nullable = true)
 |-- Review_Total_Positive_Word_Counts: integer (nullable = true)
 |-- Total_Number_of_Reviews_Reviewer_Has_Given: integer (nullable = true)
 |-- Reviewer_Score: double (nullable = true)
 |-- Tags: string (nullable = true)
 |-- days_since_review: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- lng: string (nullable = true)
 |-- Review: string (nullable = true)



In [11]:
df_cleaned.head(5)

[Row(id=14, Hotel_Address='s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands', Additional_Number_of_Scoring=194, Review_Date='7/4/2017', Average_Score=7.7, Hotel_Name='Hotel Arena', Reviewer_Nationality='Canada', Review_Total_Negative_Word_Counts=35, Total_Number_of_Reviews=1403, Review_Total_Positive_Word_Counts=15, Total_Number_of_Reviews_Reviewer_Has_Given=1, Reviewer_Score=8.8, Tags="[' Leisure trip ', ' Family with young children ', ' Large King Room ', ' Stayed 5 nights ', ' Submitted from a mobile device ']", days_since_review='30 days', lat='52.3605759', lng='4.9159683', Review='the staff in the restaurant could of been more pleasant we only visited once but that wouldn t stop us from booking again it might of just been an off night for him .  it was very good very historic building that s why I chose it'),
 Row(id=374, Hotel_Address='s Gravesandestraat 55 Oost 1092 AA Amsterdam Netherlands', Additional_Number_of_Scoring=194, Review_Date='11/3/2015', Average_Score=7.7, H

# VADER

In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


In [22]:
import numpy as np

In [6]:
vader = SentimentIntensityAnalyzer()

In [19]:
rdd_scores = df_cleaned.select("id", "Review", "Reviewer_Score").rdd.map(lambda x: (x["id"], 
                                                                       ((vader.polarity_scores(x["Review"])['compound']  ) +1)*5,
                                                                       x["Reviewer_Score"]))



#.toDF(["id", "Vader_scores", "User_scores"])
#df_scores.printSchema()

In [20]:
rdd_scores.take(10)

[(14, 7.92, 8.8),
 (374, 8.591999999999999, 9.2),
 (681, 9.8475, 8.8),
 (860, 5.0, 9.6),
 (1014, 5.0, 9.2),
 (1123, 9.643999999999998, 10.0),
 (1162, 3.4684999999999997, 5.0),
 (1318, 9.907, 9.2),
 (1375, 9.5775, 9.6),
 (1705, 8.7125, 9.2)]

In [48]:
rdd_scores.filter(lambda x: type(x[2]) != float and type(x[1]) != float ).take(200)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 33.0 failed 1 times, most recent failure: Lost task 0.0 in stage 33.0 (TID 245, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/hadoopuser/spark/python/pyspark/rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<ipython-input-19-c9fb15013d29>", line 2, in <lambda>
  File "/home/student18/env-py3/lib/pyth

In [31]:
empty_acc = (0.0,0.0)

In [32]:
def mergeValue(acc, value):
    return (acc[0] + value, acc[1] + 1)

In [33]:
def mergeAccum(acc1,acc2):
    return (acc1[0] + acc2[0], acc1[1] + acc2[1])

In [38]:
rdd_scores.map(lambda x: ( (x[1] - x[2])**2 )).aggregate(empty_acc, mergeValue, mergeAccum)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 23.0 failed 1 times, most recent failure: Lost task 0.0 in stage 23.0 (TID 233, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 174, in main
    process()
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/worker.py", line 169, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/hadoopuser/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/hadoopuser/spark/python/pyspark/rdd.py", line 932, in func
    for obj in iterator:
  File "<ipython-input-19-c9fb15013d29>", line 2, in <lambda>
  File "/home/student18/env-py3/lib/python3

In [23]:
bins = np.array([1,2,3,4,5])

In [29]:
def digitize(score):
    x = float(score)/2
    return int(np.digitize(x,bins,right=True))+1

In [30]:
rdd_scores.map(lambda x: (  x[0], digitize(x[1]), digitize(x[2]))).take(10)

[(14, 4, 5),
 (374, 5, 5),
 (681, 5, 5),
 (860, 3, 5),
 (1014, 3, 5),
 (1123, 5, 5),
 (1162, 2, 3),
 (1318, 5, 5),
 (1375, 5, 5),
 (1705, 5, 5)]

In [ ]:

from sklearn.metrics import accuracy_score
pr_score = accuracy_score(X, Y)
print("accuracy:", pr_score)